In [6]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'
SINCE_DATE = int(datetime(2017, 1, 1).timestamp())
UNTIL_DATE = int(datetime.now().timestamp())

# URLs for fetching data
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/mvrv'
]

# List of all assets (split into chunks for readability)
# List of all assets
ASSETS = [
    'BTC', 'ETH', 'LTC', '1INCH', 'AAVE', 'ABT', 'ACH', 'ACX', 'ADP', 'ADS', 'AE', 'AGIX', 
    'AGLD', 'AGRS', 'AHT', 'AIAT', 'AIOZ', 'AIPAD', 'AIT', 'ALCX', 'ALD', 'ALEPH', 'ALI', 
    'ALICE', 'ALKI', 'ALPH', 'ALPHA', 'AMB', 'AMO', 'AMP', 'AMPL', 'ANGLE', 'ANKR', 'ANT', 
    'APE', 'APEX', 'APFC', 'API3', 'APRS', 'AQT', 'ARIX', 'ARKM', 'ARPA', 'ASD', 'ASIA', 
    'AST', 'ASTO', 'ATA', 'AUCTION', 'AUDIO', 'AURORA', 'AVA', 'AVAX', 'AXGT', 'AXL', 'AXS', 
    'B2M', 'BABYDOGE', 'BAD', 'BADGER', 'BAL', 'BAND', 'BAT', 'BAX', 'BEAMMW', 'BEL', 'BEPRO', 
    'BETA', 'BFC', 'BGB', 'BICO', 'BIGTIME', 'BITRUE', 'BKN', 'BLENDR', 'BLUR', 'BMEX', 'BMX', 
    'BNT', 'BOA', 'BOBA', 'BOBT', 'BOND', 'BONE', 'BONK', 'BOO', 'BORA', 'BORG', 'BOSON', 
    'BOTTO', 'BREED', 'BRG', 'BRISE', 'BROCK', 'BRWL', 'BTM', 'BUSD', 'BXX', 'BZR', 'BZRX', 
    'C98', 'CAGA', 'CAH', 'CAKE', 'CANTO', 'CAPS', 'CAST', 'CAW', 'CBETH', 'CBK', 'CBY', 
    'CEEK', 'CEL', 'CELL', 'CELR', 'CERE', 'CET', 'CGPT', 'CHEQ', 'CHESS', 'CHEX', 'CHR', 
    'CHRP', 'CHZ', 'CLV', 'COMAI', 'COMBO', 'COMP', 'COPI', 'CORGIAI', 'COTI', 'COVAL', 'COW', 
    'CPOOL', 'CQT', 'CRE', 'CREAM', 'CREDI', 'CREO', 'CRO', 'CRTS', 'CRU', 'CRV', 'CSIX', 
    'CSWAP', 'CTA', 'CTC', 'CTSI', 'CTX', 'CTXC', 'CUDOS', 'CULT', 'CVC', 'CVP', 'CVX', 
    'CVXCRV', 'CWEB', 'DAG', 'DAI', 'DAO', 'DAR', 'DATA', 'DC', 'DDX', 'DEAI', 'DEFIT', 'DEGO', 
    'DENT', 'DEP', 'DERC', 'DEXE', 'DEXT', 'DF', 'DFI', 'DHT', 'DIA', 'DIONE', 'DKA', 'DMAIL', 
    'DMTR', 'DNT', 'DODO', 'DOGENFT', 'DOLA', 'DOMI', 'DOSE', 'DPI', 'DRGN', 'DUSK', 'DUST', 
    'ECOX', 'EDEN', 'EDU', 'EFI', 'EJS', 'EL', 'ELA', 'ELF', 'ELON', 'ENA', 'ENG', 'ENJ', 
    'ENQAI', 'ENS', 'EPIK', 'ETHFI', 'EUL', 'EURS', 'EURT', 'FARM', 'FCT', 'FDUSD', 'FEI', 
    'FER', 'FET', 'FI', 'FIS', 'FLEX', 'FLIP', 'FLOKI', 'FLX', 'FOOM', 'FOR', 'FORT', 'FOX', 
    'FRAX', 'FRM', 'FRONT', 'FTM', 'FTT', 'FUN', 'FUSE', 'FX', 'FXS', 'GAL', 'GALA', 'GEEQ', 
    'GELATO', 'GEOJ', 'GFI', 'GHST', 'GLM', 'GMM', 'GMTT', 'GNO', 'GODS', 'GOG', 'GPU', 'GRT', 
    'GT', 'GUSD', 'HAI', 'HARRYP', 'HEART', 'HEGIC', 'HELLO', 'HEMULE', 'HEZ', 'HFT', 'HOPR', 
    'HOT', 'HT', 'HVH', 'IAG', 'ICHI', 'ID', 'IDEX', 'ILV', 'IMGNAI', 'IMX', 'INDEX', 'INJ', 
    'INSP', 'INV', 'IOTX', 'IPOR', 'IQ', 'ISKR', 'ISP', 'JOECOIN', 'JPEG', 'KAI', 'KARATE', 
    'KATA', 'KCS', 'KEEP', 'KEY', 'KISHU', 'KNDX', 'KOMPETE', 'KP3R', 'KRL', 'LADYS', 'LAI', 
    'LAMB', 'LBA', 'LDO', 'LEO', 'LEVER', 'LINA', 'LINK', 'LIT', 'LITH', 'LM', 'LMWR', 'LOKA', 
    'LON', 'LOOKS', 'LPT', 'LQTY', 'LRC', 'LSETH', 'LSK', 'LSS', 'LUNC', 'LYXE', 'MAGIC', 
    'MAHA', 'MAN', 'MANA', 'MANTLE', 'MAP', 'MARSH', 'MATH', 'MATIC', 'MAVIA', 'MBL', 'MC', 
    'MCADE', 'MCB', 'MCRT', 'META', 'METH', 'METIS', 'MFT', 'MIN', 'MIR', 'MIX', 'MKR', 'MLN', 
    'MLT', 'MMX', 'MOC', 'MOG', 'MPL', 'MTA', 'MTD', 'MTL', 'MTLX', 'MUBI', 'MUSE', 'MUSIC', 
    'MVL', 'MX', 'MXC', 'MYRIA', 'MYTH', 'NAVI', 'NCT', 'NDX', 'NEAR', 'NEST', 'NEXO', 'NFT', 
    'NFTX', 'NIZA', 'NKN', 'NMR', 'NOIA', 'NRG', 'NSURE', 'NULS', 'NUM', 'NVIR', 'NWC', 'NXRA', 
    'NYM', 'OAX', 'OBSR', 'OCEAN', 'OCT', 'OGN', 'OGV', 'OHMV2', 'OKB', 'OLAS', 'OM', 'OMG', 
    'OMI', 'OOE', 'OOKI', 'OPEN', 'OPSEC', 'OPTI', 'OPUL', 'ORAI', 'ORB', 'ORBS', 'ORDS', 
    'ORN', 'OSAK', 'OVR', 'OX', 'OXT', 'OXY', 'PAAL', 'PANDORA', 'PARI', 'PAW', 'PAY', 'PDA', 
    'PEAS', 'PEOPLE', 'PEPE', 'PEPE2', 'PEPECOIN', 'PERP', 'PHA', 'PIB', 'PICKLE', 'PKF', 
    'PLU', 'PNK', 'PNT', 'POL', 'POLA', 'POLS', 'POLY', 'POND', 'POOH', 'PORK', 'PORT3', 
    'POWR', 'PPT', 'PRE', 'PRIME', 'PRNT', 'PROM', 'PROPC', 'PROS', 'PRQ', 'PSP', 'PSPS', 
    'PSTAKE', 'PSWAP', 'PTU', 'PUNDIX', 'PUSH', 'PYR', 'PYTH', 'PYUSD', 'QASH', 'QKC', 'QNT', 
    'QORPO', 'QTCON', 'RACA', 'RAD', 'RADAR', 'RAI', 'RAIL', 'RARE', 'RARI', 'RBN', 'RBX', 
    'RDN', 'RDNT', 'REEF', 'REN', 'REP', 'REQ', 'RETH', 'REVV', 'REZ', 'RING', 'RLB', 'RLC', 
    'RMRK', 'RNDR', 'ROOK', 'ROUTE', 'RPL', 'RSC', 'RSR', 'RSS3', 'RUNE', 'RVF', 'SAITA', 
    'SAITO', 'SAMA', 'SAND', 'SAVM', 'SD', 'SDAO', 'SDEX', 'SEAM', 'SENATE', 'SFP', 'SFRXETH', 
    'SFUND', 'SHFT', 'SHIB', 'SHIDO', 'SHRAP', 'SHX', 'SIDESHIFT', 'SIDUS', 'SILO', 'SIS', 
    'SKEB', 'SKEY', 'SKL', 'SLN', 'SMURFCATETH', 'SNT', 'SNX', 'SOFI', 'SOLVE', 
    'SOMNIUM', 'SORA', 'SOV', 'SPA', 'SPOOL', 'SQUIDGROW', 'SRM', 'SSV', 'STAKE', 'STARL', 
    'STAT', 'STBU', 'STETH', 'STG', 'STMX', 'STORJ', 'STOS', 'STPT', 'STRP', 'STRUMP', 'SUKU', 
    'SUPER', 'SURE', 'SUSD', 'SUSHI', 'SWAP', 'SWASH', 'SWFTC', 'SXP', 'SYLO', 'SYN', 'TARA', 
    'TBTC', 'TEL', 'TENET', 'TET', 'THALES', 'THOR', 'TIME', 'TKX', 'TLM', 'TLOS', 'TOKE', 
    'TOMI', 'TORN', 'TOWER', 'TPT', 'TRAC', 'TRIBE', 'TRUF', 'TRUMP', 'TRVL', 'TRX', 'TRYB', 
    'TSUKA', 'TT', 'TURBOT', 'TUSD', 'TVK', 'UBT', 'UDS', 'UFI', 'UFO', 'UFT', 'UMA', 'UNCX', 
    'UNFI', 'UNI', 'UNIBOT', 'UOS', 'UQC', 'USDC', 'USDD', 'USDE', 'USDP', 'USDT', 'USDY', 
    'USTC', 'UTK', 'UX', 'VALOR', 'VEGA', 'VERI', 'VEXT', 'VGX', 'VIB', 'VIDT', 'VIRTUAL', 
    'VIX', 'VMINT', 'VR', 'VRA', 'VVS', 'VXV', 'WAGMIGAMES', 'WALLET', 'WAMPL', 'WAVES', 
    'WAXP', 'WBETH', 'WBT', 'WBTC', 'WCFG', 'WETH', 'WHALE', 'WIKEN', 'WILD', 'WING', 'WISE', 
    'WLD', 'WMT', 'WNXM', 'WOJAK', 'WOO', 'WRLD', 'WRX', 'WSM', 'WXT', 'XAUT', 'XCAD', 'XCN', 
    'XDAO', 'XDB', 'XDEFI', 'XEN', 'XETA', 'XOR', 'XTM', 'XTP', 'XVS', 'XYO', 'YAM', 'YFI', 
    'YFII', 'YGG', 'YLD', 'ZCX', 'ZENT', 'ZIG', 'ZKML', 'ZRX'
]

# Print some statistics
print(f"Total number of assets: {len(ASSETS)}")
print(f"First 5 assets: {ASSETS[:5]}")
print(f"Last 5 assets: {ASSETS[-5:]}")

def fetch_glassnode_data(url, asset='BTC'):
    """
    Fetches data from Glassnode API for a specific metric and asset.
    """
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        column_name = f"{metric_name}_{asset}"
        df.columns = ['t', column_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {asset} from {url}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error processing data for {asset} from {url}: {e}")
        return None

Total number of assets: 659
First 5 assets: ['BTC', 'ETH', 'LTC', '1INCH', 'AAVE']
Last 5 assets: ['ZCX', 'ZENT', 'ZIG', 'ZKML', 'ZRX']


In [7]:

def fetch_all_data():
    """
    Fetches data for all metrics and assets and combines them into a single DataFrame.
    """
    all_dfs = []
    for metric_url in METRICS:
        for asset in ASSETS:
            #print(f"Fetching {metric_url.split('/')[-1]} for {asset}")
            df = fetch_glassnode_data(metric_url, asset)
            if df is not None:
                all_dfs.append(df)

    if not all_dfs:
        raise Exception("No data was successfully fetched")

    merged_df = all_dfs[0]
    for df in all_dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='t', how='outer')

    merged_df.set_index('t', inplace=True)
    return merged_df

try:
    final_df = fetch_all_data()
    print("Data collection complete")
    print(f"DataFrame shape: {final_df.shape}")
    print("\nColumns in the dataset:")
    print(final_df.columns.tolist())
except Exception as e:
    print(f"Error in data collection process: {e}")

Data collection complete
DataFrame shape: (2863, 659)

Columns in the dataset:
['mvrv_BTC', 'mvrv_ETH', 'mvrv_LTC', 'mvrv_1INCH', 'mvrv_AAVE', 'mvrv_ABT', 'mvrv_ACH', 'mvrv_ACX', 'mvrv_ADP', 'mvrv_ADS', 'mvrv_AE', 'mvrv_AGIX', 'mvrv_AGLD', 'mvrv_AGRS', 'mvrv_AHT', 'mvrv_AIAT', 'mvrv_AIOZ', 'mvrv_AIPAD', 'mvrv_AIT', 'mvrv_ALCX', 'mvrv_ALD', 'mvrv_ALEPH', 'mvrv_ALI', 'mvrv_ALICE', 'mvrv_ALKI', 'mvrv_ALPH', 'mvrv_ALPHA', 'mvrv_AMB', 'mvrv_AMO', 'mvrv_AMP', 'mvrv_AMPL', 'mvrv_ANGLE', 'mvrv_ANKR', 'mvrv_ANT', 'mvrv_APE', 'mvrv_APEX', 'mvrv_APFC', 'mvrv_API3', 'mvrv_APRS', 'mvrv_AQT', 'mvrv_ARIX', 'mvrv_ARKM', 'mvrv_ARPA', 'mvrv_ASD', 'mvrv_ASIA', 'mvrv_AST', 'mvrv_ASTO', 'mvrv_ATA', 'mvrv_AUCTION', 'mvrv_AUDIO', 'mvrv_AURORA', 'mvrv_AVA', 'mvrv_AVAX', 'mvrv_AXGT', 'mvrv_AXL', 'mvrv_AXS', 'mvrv_B2M', 'mvrv_BABYDOGE', 'mvrv_BAD', 'mvrv_BADGER', 'mvrv_BAL', 'mvrv_BAND', 'mvrv_BAT', 'mvrv_BAX', 'mvrv_BEAMMW', 'mvrv_BEL', 'mvrv_BEPRO', 'mvrv_BETA', 'mvrv_BFC', 'mvrv_BGB', 'mvrv_BICO', 'mvrv_BIGT

In [8]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [11]:
def plot_all_mvrv(df, initial_scale='linear', smoothing=365):
    """
    Create an interactive Plotly figure showing all MVRV ratios
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing MVRV columns
    initial_scale (str): Initial y-axis scale ('linear' or 'log')
    smoothing (int): Window size for Simple Moving Average smoothing. Set to 0 to disable.
    """
    # Get all MVRV columns
    mvrv_columns = [col for col in df.columns if col.startswith('mvrv_')]
    
    # Create a copy of the dataframe for smoothing
    if smoothing > 0:
        df_smooth = df.copy()
        for col in mvrv_columns:
            df_smooth[col] = df[col].rolling(window=smoothing, min_periods=1).mean()
    else:
        df_smooth = df
    
    # Create the figure
    fig = go.Figure()
    
    # Generate colors using plotly's built-in color sequences
    import plotly.express as px
    colors = px.colors.qualitative.Set3
    color_idx = 0
    num_colors = len(colors)
    
    # Add watermark using annotation
    fig.add_annotation(
        text="_cryptovizart",
        xref="paper",
        yref="paper",
        x=0.5,
        y=0.5,
        showarrow=False,
        font=dict(size=100, color="rgba(200,200,200,0.2)"),
        textangle=0
    )
    
    # Add traces for each MVRV
    for column in mvrv_columns:
        crypto_name = column.replace('mvrv_', '')
        
        # Set color and line style for the trace
        if crypto_name == 'BTC':
            color = '#000000'  # Dark black for Bitcoin
            line_style = 'dash'  # Dashed line for Bitcoin
        else:
            color = colors[color_idx % num_colors]
            line_style = 'solid'  # Solid line for others
            color_idx += 1
        
        fig.add_trace(
            go.Scatter(
                x=df_smooth.index,
                y=df_smooth[column],
                name=crypto_name,
                line=dict(
                    color=color,
                    dash=line_style,
                    width=2  # Make lines more visible
                ),
                mode='lines',
                connectgaps=False  # Don't connect gaps in the data
            )
        )
    
    # Add y=1 reference line
    fig.add_hline(
        y=1,
        line_dash="dash",
        line_color="gray",
        opacity=0.7,
        name="MVRV = 1"
    )
    
    # Update layout
    fig.update_layout(
        title=f'Cryptocurrency MVRV Ratios Over Time (SMA-{smoothing})' if smoothing > 0 
              else 'Cryptocurrency MVRV Ratios Over Time',
        xaxis_title='Date',
        yaxis_title='MVRV Ratio',
        template='none',  # Remove template to start with clean slate
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white',  # Set paper background to white
        hovermode='x unified',
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.05
        ),
        # Make the figure responsive
        height=800,
        width=1200,
        margin=dict(r=150),  # Add right margin for legend
        # Set initial y-axis scale
        yaxis_type=initial_scale,
        # Remove gridlines
        xaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=True,
            linewidth=1,
            linecolor='black',
            mirror=True
        ),
        yaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=True,
            linewidth=1,
            linecolor='black',
            mirror=True
        )
    )
    
    # Add range slider
    fig.update_xaxes(rangeslider_visible=True)
    
    return fig

In [ ]:

# Usage example:
combined_fig = plot_all_mvrv(final_df)
# Display the figures
combined_fig.show()
